In [217]:
import numpy as np
from scipy.optimize import minimize

In [218]:
class Matrix:
    __array_priority__ = 1
    
    def __init__(self, matrix):
        self.m, self.n = matrix.shape
        self.matrix = matrix.copy()
        self.data = [(i, j, matrix[i][j]) for i in range(self.m) for j in range(self.n) if matrix[i][j]]
        
    def T(self):
        res = np.zeros((self.n, self.m))
        for i, j, val in self.data:
            res[j][i] = val
        return Matrix(res)
    
    def __matmul__(self, other):
        assert self.n == other.shape[0]
        res = np.zeros(self.m)
        for i, j, val in self.data:
            res[i] += other[j] * val
        return res
        
    def __rmatmul__(self, other):
        assert self.m == other.shape[0]
        res = np.zeros(self.n)
        for i, j, val in self.data:
            res[j] += other[i] * val
        return res

In [219]:
matrix = np.array(
    [
        [1., 4., 0.], 
        [2., 1., 0.], 
        [0., 1., 0.5]
    ]
)
M = Matrix(matrix)
v = np.array([1., 2., 3.])

assert np.all(M.T().matrix == matrix.T)
assert np.all(Matrix(matrix) @ v == matrix @ v)
assert np.all(v @ Matrix(matrix) == v @ matrix)

In [220]:
def ConjGrad(A, b):
    x = np.zeros(A.m)
    AT = A.T()
    b = AT @ b
    v = AT @ (A @ x) - b
    v_norm = np.dot(v, v)
    d = v
    
    for i in range(len(b)):
        Ad = AT @ (A @ d)
        alpha = v_norm / (d @ Ad)
        v = v - alpha * Ad
        x = x - alpha * d
        new_v_norm = np.dot(v, v)
        d = v + (new_v_norm / v_norm) * d
        v_norm = new_v_norm
    return x

In [222]:
np.random.seed(2)
for i in range(50):
    m = 5
    A = Matrix(np.array(np.random.rand(m, m)))
    b = np.random.rand(m)
    
    assert np.abs(np.linalg.norm(A @ ConjGrad(A, b) - b) - \
                  minimize(lambda x: np.linalg.norm(A @ x - b), np.zeros(m)).fun) < 1e-5